In [132]:
import datetime
import pytz

import matplotlib.pyplot as plt
import mplcyberpunk

from pathlib import Path
import pandas as pd
import numpy as np
from astropy.io import fits
import astropy.wcs as fitswcs
import astropy.units as u
from astropy.coordinates import EarthLocation, SkyCoord, AltAz, get_moon, Angle
from astropy.time import Time
from astropy.table import Table
import astroplan
from specutils.spectra.spectrum1d import Spectrum1D
from astropy.visualization import quantity_support
from specutils.manipulation import FluxConservingResampler

from sklearn.decomposition import PCA

_ = quantity_support()
plt.style.use("cyberpunk")
%matplotlib widget

In [103]:
df = pd.read_csv("sky_spectra.csv")
moonfree = df[(df['moon_alt'] < -10) & (df['disperser'] == '270_gpm')]
moonfree

,file,ut,alt,az,moon_alt,moon_az,moon_ill,disperser
1,./2004.0413/skysub_a2034.bg_1_comb.fits,2004-04-13T08:25:25,81.18,75.47,-17.42,107.26,37,270_gpm
3,./2004.0413/skysub_F2.20.bg_1_comb.fits,2004-04-13T05:53:13,56.71,278.43,-47.97,91.12,38,270_gpm
4,./2004.0414/skysub_F2.bg_1_comb.fits,2004-04-14T05:11:49,65.04,274.96,-65.06,67.04,28,270_gpm
5,./2004.0414/skysub_F2.bg_2_comb.fits,2004-04-14T07:00:59,41.07,284.86,-43.32,87.36,27,270_gpm
6,./2004.0415/skysub_F2.bg_3_comb.fits,2004-04-15T05:48:01,55.37,278.13,-64.16,53.43,19,270_gpm
...,...,...,...,...,...,...,...,...
4126,./2020.1013/skysub_m31_pne_20_9_comb.fits,2020-10-13T08:16:47,66.13,300.54,-16.84,60.23,17,270_gpm
4129,./2020.1024/skysub_jiang_c2_1_comb.fits,2020-10-24T12:04:06,65.47,80.09,-59.19,282.18,60,270_gpm
4132,./2020.1024/skysub_Rank13_1_comb.fits,2020-10-24T08:14:37,50.94,204.84,-13.27,252.44,58,270_gpm
4133,./2020.1025/skysub_jiang_c2_1_comb.fits,2020-10-25T11:29:55,59.13,78.40,-41.03,273.43,69,270_gpm


In [104]:
new_disp_grid = np.arange(3850, 6800, 2) * u.AA
fluxcon = FluxConservingResampler()

In [105]:
fluxes = []
for i, r in moonfree.iterrows():
    sp = Spectrum1D.read(r['file'])
    trim_sp = fluxcon(sp, new_disp_grid)
    fluxes.append(trim_sp.flux)

In [168]:
fluxes = np.array(fluxes)
fits.writeto("sky_fluxes.fts", fluxes)

In [133]:
plt.figure(figsize=[15,8])
plt.yscale('log')
plt.step(new_disp_grid, np.mean(fluxes, axis=0))
plt.step(new_disp_grid, np.std(fluxes, axis=0))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [134]:
plt.figure(figsize=[15,8])
#plt.yscale('log')
plt.step(new_disp_grid, np.mean(fluxes, axis=0)/np.std(fluxes, axis=0))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [135]:
plt.figure(figsize=[15,8])
#plt.yscale('log')
plt.step(new_disp_grid, 2.5 * np.log10(fluxes[-1, :] / fluxes[2200, :]))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [136]:
pca = PCA(n_components=16).fit(fluxes)

In [137]:
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [138]:
components = pca.transform(fluxes)
filtered = pca.inverse_transform(components)

In [139]:
components.shape

(2310, 16)

In [140]:
filtered.shape

(2310, 1475)

In [178]:
plt.figure(figsize=[15,8])
#plt.yscale('log')
plt.step(new_disp_grid, fluxes[2000, :])
plt.step(new_disp_grid, fluxes[2000, :] - filtered[2000, :])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [169]:
features = Table.read("features.csv")
plt.figure(figsize=[25,16])
#plt.yscale('log')
#for i in range(5):
    #plt.step(new_disp_grid, pca_sm.components_[i], label=f"Component {i}")
i = 1
plt.step(new_disp_grid, pca.components_[i], label=f"Component {i}")
txt_y = 1.02 * np.max(pca.components_[i])
for f in features:
    if f['low wave'] != f['high wave']:
        plt.axvspan(f['low wave'], f['high wave'], color='r', alpha=0.1)
        txt_x = 0.5*(f['low wave'] + f['high wave'])
        plt.text(txt_x, 1.05*txt_y, f['name'], horizontalalignment='center', verticalalignment='top')
    else:
        x = f['low wave']
        plt.text(x, txt_y, f"{f['name']} {x}", rotation='vertical', fontsize='medium', horizontalalignment='right', verticalalignment='top')
        plt.axvline(x, color='black', alpha=0.2)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
components[-1, :]

array([-0.97426734,  1.3280089 ,  1.21197703, -2.11753905, -1.09568555,
        0.04327511, -0.02283311,  0.43418767,  0.37909889,  0.50991424,
       -0.12827532,  0.0482443 , -0.07777952, -0.12138593,  0.05516883,
       -0.05339013,  0.04032064, -0.02323196, -0.00341229,  0.03711213,
        0.01279714,  0.2873275 ,  0.4125028 ,  0.28903412])

In [90]:
components[0, :]

array([-1.02591806e+00, -2.08063226e-01, -2.29464562e+00,  4.47705699e-01,
       -1.04044543e+00, -1.23337220e-01, -1.57060074e-02, -1.47131398e-01,
       -1.50833537e-01,  3.51521293e-02, -1.27162334e-02, -6.86495993e-04,
       -2.22179677e-03, -2.84325139e-02, -4.79068319e-03,  7.01374762e-03,
        8.85344407e-04, -5.12641225e-02,  7.96013981e-03,  6.29171426e-02,
        1.07075937e-02,  5.93821801e-03, -5.65231762e-02, -9.73743154e-02])

In [172]:
np.arange(16).reshape(4, 4).flatten()[3]

3

In [177]:
fig, axs = plt.subplots(4, 4, figsize=[25,16], sharex=True)
for i, ax in enumerate(axs.flat):
    ax.step(new_disp_grid, pca.components_[i])
    ax.set_title(f"Component {i}")
    ax.set_ylabel("counts/s")
    ax.set_xlabel("Wavelength (A)")
    ax.label_outer()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …